# PV: Solar production on south- vs. west-facing rooftop and how solar systems align with 4CP events in Texas

This notebook will explore solar generation around the ERCOT 4CP events and compare West vs South facing solar generation during those events.

The ERCOT 4CP events are the 15-minute ERCOT grid peak events for each month in June, July, August and September.

ERCOT uses each large customer’s (including municipal utilities) total energy demand during the 4CP periods in the previous year as the basis for charges in the current year.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
#Read pre-processed data for June 2018 through August 2018
df2 = pd.read_csv('/shared/JupyterHub-Examples-Data/pv/pv_south_vs_west.zip',compression='zip')
df2

In [ ]:
# Checking completeness of data. We will consider only those homes that have 100% data for the 4CP dates
data = df2.groupby(['dataid']).size().reset_index(name='counts')
data['perc'] = (data['counts']/256)*100
ndata = data[data['perc'] == 100]
final_dataids = ndata['dataid']

df3=df2.loc[df2['dataid'].isin(final_dataids)]
df3.count()


In [ ]:
dataids_list = df3['dataid'].unique()
len(dataids_list)

In [ ]:
# drop any missing values
df3 = df3.dropna()

In [ ]:
# convert localminute to pandas datetime type
df3['datetime'] = pd.to_datetime(df3['localminute'])

# and set as index
df3 = df3.set_index('datetime')
df3

In [ ]:
# group by month and dataid and take the mean of solar, grid, and usage within those groups
grouped = df3.groupby([pd.Grouper(freq='M'), 'dataid']).mean()
grouped

In [ ]:
# read in the list of homes and direction that they face
df = pd.read_csv('/shared/JupyterHub-Examples-Data/pv/pv_south_vs_west_homes.zip',compression='zip')
df

In [ ]:
# map pv direction onto dataids with a merge after resetting the index
grouped = grouped.reset_index()
grouped = grouped.merge(df, how='left', left_on='dataid', right_on='dataid')
grouped

In [ ]:
# reindex by the date
grouped = grouped.set_index('datetime')
grouped

In [ ]:
# regroup by year and pv_panel_direction and average the values
year_west_vs_south = (grouped.groupby([pd.Grouper(freq='Y'),'pv_panel_direction']).mean())

# we don't need a mean of the dataids, so we can drop that column now
year_west_vs_south = year_west_vs_south.drop(columns=['dataid'])
year_west_vs_south

In [ ]:
# now just drop unneeded columns and rearrange them and simplify the view
year_west_vs_south = year_west_vs_south.reset_index()
year_west_vs_south['year'] = pd.DatetimeIndex(year_west_vs_south['datetime']).year
year_west_vs_south = year_west_vs_south[['year', 'pv_panel_direction','solar', 'grid', 'usage']]
year_west_vs_south = year_west_vs_south.set_index('year')
year_west_vs_south

# Result: West Facing Solar Wins!
## Despite the higher in-home usage (green bars, 'usage') in the west facing houses the solar production is so high (blue bars, 'solar') that it is still using less power from the grid (orage bars, 'grid') in West-facing vs South-facing home in each paired year. In most cases here, grid is even negative meaning that the West-facing homes are putting power back on the grid.

In [ ]:
from matplotlib.pyplot import figure
plot = year_west_vs_south.plot(kind='bar',figsize=(25,15), title="Solar production, Net Grid Usage (home usage from the grid minus solar production), and Home Usage During ERCOT 4CP events")
labels = plot.set_xticklabels(['2016-South', '2016-West', '2017-South', '2017-West', '2018-South', '2018-West', '2019-South', '2019-West'])
ylabel = plot.set_ylabel('Usage/Production in kW')
xlabel = plot.set_xlabel('South vs West Facing Year')